# Day 06: Model Deployment for Trading Systems

## Production ML Week - Deploying Quantitative Models

This notebook covers the critical aspects of deploying machine learning models for trading systems in production environments.

### Learning Objectives:
1. **Model Serialization** - Save and load models using joblib, pickle, and ONNX
2. **REST API Development** - Build Flask/FastAPI services for model inference
3. **Real-time Pipelines** - Create low-latency prediction pipelines
4. **Batch Processing** - Implement efficient batch prediction systems
5. **Monitoring & Logging** - Track model performance and detect drift
6. **Model Versioning** - Use MLflow for experiment tracking and model registry
7. **Containerization** - Package models with Docker for consistent deployment
8. **Deployment Strategies** - Implement blue-green and canary deployments

### Production Considerations for Trading:
- **Latency Requirements**: Sub-millisecond for HFT, seconds for swing trading
- **Reliability**: 99.99% uptime requirements
- **Scalability**: Handle market volatility spikes
- **Compliance**: Audit trails and model explainability

## 1. Import Required Libraries

In [1]:
# Core Libraries
import numpy as np
import pandas as pd
import json
import os
import sys
import time
import pickle
import joblib
import hashlib
import logging
from datetime import datetime, timedelta
from typing import Dict, List, Any, Optional, Tuple, Union
from dataclasses import dataclass, field, asdict
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Machine Learning
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# API & Web Frameworks (conceptual - typically run separately)
# from flask import Flask, request, jsonify
# from fastapi import FastAPI, HTTPException
# import uvicorn

# Monitoring & Logging
import logging
from collections import deque
import threading
from queue import Queue

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Set plotting style
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger('TradingModelDeployment')

print("✅ Libraries imported successfully")
print(f"📅 Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

✅ Libraries imported successfully
📅 Date: 2026-01-24 02:15:14


## 2. Create Sample Trading Model for Deployment

First, let's create and train a sample trading model that we'll use throughout this deployment tutorial.

In [2]:
def generate_trading_features(n_samples: int = 10000) -> pd.DataFrame:
    """Generate synthetic trading data with features."""
    np.random.seed(42)
    
    # Price data simulation
    returns = np.random.randn(n_samples) * 0.02
    price = 100 * np.exp(np.cumsum(returns))
    
    # Technical indicators as features
    df = pd.DataFrame({
        'price': price,
        'returns': returns,
        'volatility_20': pd.Series(returns).rolling(20).std().fillna(0.01),
        'momentum_10': pd.Series(price).pct_change(10).fillna(0),
        'momentum_20': pd.Series(price).pct_change(20).fillna(0),
        'rsi': 50 + 30 * np.tanh(returns * 10),  # Simplified RSI
        'volume': np.random.exponential(1000000, n_samples),
        'spread': np.random.exponential(0.001, n_samples),
        'hour': np.random.randint(9, 17, n_samples),
        'day_of_week': np.random.randint(0, 5, n_samples),
    })
    
    # Add more complex features
    df['volume_ma'] = df['volume'].rolling(20).mean().fillna(df['volume'].mean())
    df['volume_ratio'] = df['volume'] / df['volume_ma']
    df['price_ma_20'] = df['price'].rolling(20).mean().fillna(df['price'].iloc[0])
    df['price_ma_50'] = df['price'].rolling(50).mean().fillna(df['price'].iloc[0])
    df['ma_crossover'] = (df['price_ma_20'] > df['price_ma_50']).astype(int)
    
    # Target: 1 if next return > 0.1%, else 0
    future_returns = df['returns'].shift(-1).fillna(0)
    df['target'] = (future_returns > 0.001).astype(int)
    
    return df

# Generate training data
trading_data = generate_trading_features(10000)
print(f"📊 Generated {len(trading_data)} samples with {len(trading_data.columns)} features")
trading_data.head()

📊 Generated 10000 samples with 16 features


,price,returns,volatility_20,momentum_10,momentum_20,rsi,volume,spread,hour,day_of_week,volume_ma,volume_ratio,price_ma_20,price_ma_50,ma_crossover,target
0,100.998379,0.009934,0.01,0.0,0.0,52.970519,387510.029736,0.000188,16,1,1.026734e+06,0.377420,100.998379,100.998379,0,0
1,100.719476,-0.002765,0.01,0.0,0.0,49.170626,108720.815639,0.000426,15,4,1.026734e+06,0.105890,100.998379,100.998379,0,1
2,102.032660,0.012954,0.01,0.0,0.0,53.864540,74516.181324,0.003510,12,1,1.026734e+06,0.072576,100.998379,100.998379,0,1
3,105.188455,0.030461,0.01,0.0,0.0,58.865661,99020.630710,0.001518,16,3,1.026734e+06,0.096442,100.998379,100.998379,0,0
4,104.697002,-0.004683,0.01,0.0,0.0,48.596106,874355.823733,0.000332,16,3,1.026734e+06,0.851590,100.998379,100.998379,0,0


In [3]:
# Define feature columns
FEATURE_COLUMNS = [
    'returns', 'volatility_20', 'momentum_10', 'momentum_20', 'rsi',
    'volume_ratio', 'ma_crossover', 'spread', 'hour', 'day_of_week'
]

# Prepare features and target
X = trading_data[FEATURE_COLUMNS].values
y = trading_data['target'].values

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create and train a pipeline with preprocessing
model_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', RandomForestClassifier(
        n_estimators=100,
        max_depth=10,
        min_samples_split=20,
        random_state=42,
        n_jobs=-1
    ))
])

# Train the model
model_pipeline.fit(X_train, y_train)

# Evaluate
train_acc = model_pipeline.score(X_train, y_train)
test_acc = model_pipeline.score(X_test, y_test)
y_pred = model_pipeline.predict(X_test)

print(f"✅ Model Training Complete")
print(f"   Train Accuracy: {train_acc:.4f}")
print(f"   Test Accuracy:  {test_acc:.4f}")
print(f"   Precision:      {precision_score(y_test, y_pred):.4f}")
print(f"   Recall:         {recall_score(y_test, y_pred):.4f}")
print(f"   F1 Score:       {f1_score(y_test, y_pred):.4f}")

✅ Model Training Complete
   Train Accuracy: 0.7742
   Test Accuracy:  0.5040
   Precision:      0.4680
   Recall:         0.2437
   F1 Score:       0.3205


## 3. Model Serialization Functions

Model serialization is critical for deployment. We'll implement robust serialization with metadata tracking, versioning, and integrity verification.

In [4]:
@dataclass
class ModelMetadata:
    """Metadata container for deployed models."""
    model_name: str
    version: str
    created_at: str
    model_type: str
    feature_names: List[str]
    target_name: str
    training_metrics: Dict[str, float]
    hyperparameters: Dict[str, Any]
    checksum: str = ""
    description: str = ""
    author: str = ""
    
    def to_dict(self) -> Dict:
        return asdict(self)
    
    @classmethod
    def from_dict(cls, data: Dict) -> 'ModelMetadata':
        return cls(**data)


class ModelSerializer:
    """
    Production-grade model serialization with metadata and versioning.
    
    Features:
    - Multiple serialization formats (joblib, pickle)
    - Checksum verification for integrity
    - Metadata tracking
    - Version management
    """
    
    def __init__(self, model_dir: str = "./models"):
        self.model_dir = Path(model_dir)
        self.model_dir.mkdir(parents=True, exist_ok=True)
        logger.info(f"ModelSerializer initialized with directory: {self.model_dir}")
    
    def _compute_checksum(self, filepath: Path) -> str:
        """Compute SHA256 checksum for model file."""
        sha256_hash = hashlib.sha256()
        with open(filepath, "rb") as f:
            for byte_block in iter(lambda: f.read(4096), b""):
                sha256_hash.update(byte_block)
        return sha256_hash.hexdigest()
    
    def save_model(
        self,
        model: Any,
        model_name: str,
        version: str,
        feature_names: List[str],
        training_metrics: Dict[str, float],
        hyperparameters: Dict[str, Any],
        target_name: str = "target",
        format: str = "joblib",
        description: str = "",
        author: str = ""
    ) -> Tuple[Path, ModelMetadata]:
        """
        Save model with metadata and versioning.
        
        Parameters:
        -----------
        model : trained model object
        model_name : name identifier for the model
        version : semantic version string (e.g., "1.0.0")
        feature_names : list of feature column names
        training_metrics : dict of training metrics
        hyperparameters : dict of model hyperparameters
        format : "joblib" or "pickle"
        
        Returns:
        --------
        Tuple of (model_path, metadata)
        """
        # Create version directory
        version_dir = self.model_dir / model_name / version
        version_dir.mkdir(parents=True, exist_ok=True)
        
        # Define file paths
        model_filename = f"{model_name}_{version}.{'joblib' if format == 'joblib' else 'pkl'}"
        model_path = version_dir / model_filename
        metadata_path = version_dir / "metadata.json"
        
        # Save model
        if format == "joblib":
            joblib.dump(model, model_path)
        else:
            with open(model_path, 'wb') as f:
                pickle.dump(model, f)
        
        # Compute checksum
        checksum = self._compute_checksum(model_path)
        
        # Create metadata
        metadata = ModelMetadata(
            model_name=model_name,
            version=version,
            created_at=datetime.now().isoformat(),
            model_type=type(model).__name__,
            feature_names=feature_names,
            target_name=target_name,
            training_metrics=training_metrics,
            hyperparameters=hyperparameters,
            checksum=checksum,
            description=description,
            author=author
        )
        
        # Save metadata
        with open(metadata_path, 'w') as f:
            json.dump(metadata.to_dict(), f, indent=2)
        
        logger.info(f"Model saved: {model_path}")
        logger.info(f"Checksum: {checksum[:16]}...")
        
        return model_path, metadata
    
    def load_model(
        self,
        model_name: str,
        version: str = "latest",
        verify_checksum: bool = True
    ) -> Tuple[Any, ModelMetadata]:
        """
        Load model with integrity verification.
        
        Parameters:
        -----------
        model_name : name identifier for the model
        version : specific version or "latest"
        verify_checksum : whether to verify file integrity
        
        Returns:
        --------
        Tuple of (model, metadata)
        """
        model_base_dir = self.model_dir / model_name
        
        if not model_base_dir.exists():
            raise FileNotFoundError(f"Model '{model_name}' not found")
        
        # Get version directory
        if version == "latest":
            versions = sorted([d.name for d in model_base_dir.iterdir() if d.is_dir()])
            if not versions:
                raise FileNotFoundError(f"No versions found for model '{model_name}'")
            version = versions[-1]
            logger.info(f"Loading latest version: {version}")
        
        version_dir = model_base_dir / version
        
        # Load metadata
        metadata_path = version_dir / "metadata.json"
        with open(metadata_path, 'r') as f:
            metadata = ModelMetadata.from_dict(json.load(f))
        
        # Find model file
        model_files = list(version_dir.glob(f"{model_name}_{version}.*"))
        if not model_files:
            raise FileNotFoundError(f"Model file not found in {version_dir}")
        
        model_path = model_files[0]
        
        # Verify checksum
        if verify_checksum:
            computed_checksum = self._compute_checksum(model_path)
            if computed_checksum != metadata.checksum:
                raise ValueError("Model file checksum mismatch! File may be corrupted.")
            logger.info("✅ Checksum verified")
        
        # Load model
        if model_path.suffix == '.joblib':
            model = joblib.load(model_path)
        else:
            with open(model_path, 'rb') as f:
                model = pickle.load(f)
        
        logger.info(f"Model loaded: {model_name} v{version}")
        return model, metadata
    
    def list_versions(self, model_name: str) -> List[Dict]:
        """List all versions of a model with their metadata."""
        model_base_dir = self.model_dir / model_name
        
        if not model_base_dir.exists():
            return []
        
        versions = []
        for version_dir in sorted(model_base_dir.iterdir()):
            if version_dir.is_dir():
                metadata_path = version_dir / "metadata.json"
                if metadata_path.exists():
                    with open(metadata_path, 'r') as f:
                        metadata = json.load(f)
                    versions.append({
                        'version': version_dir.name,
                        'created_at': metadata.get('created_at'),
                        'metrics': metadata.get('training_metrics')
                    })
        
        return versions

# Initialize serializer
serializer = ModelSerializer(model_dir="./deployed_models")
print("✅ ModelSerializer initialized")

2026-01-24 02:15:17,014 - TradingModelDeployment - INFO - ModelSerializer initialized with directory: deployed_models


✅ ModelSerializer initialized


In [5]:
# Save our trained model
training_metrics = {
    'accuracy': float(test_acc),
    'precision': float(precision_score(y_test, y_pred)),
    'recall': float(recall_score(y_test, y_pred)),
    'f1_score': float(f1_score(y_test, y_pred))
}

hyperparameters = {
    'n_estimators': 100,
    'max_depth': 10,
    'min_samples_split': 20
}

model_path, metadata = serializer.save_model(
    model=model_pipeline,
    model_name="trading_signal_classifier",
    version="1.0.0",
    feature_names=FEATURE_COLUMNS,
    training_metrics=training_metrics,
    hyperparameters=hyperparameters,
    description="Random Forest classifier for trading signal prediction",
    author="Quant Team"
)

print(f"\n📁 Model saved to: {model_path}")
print(f"📋 Metadata:")
print(json.dumps(metadata.to_dict(), indent=2, default=str))

2026-01-24 02:15:17,162 - TradingModelDeployment - INFO - Model saved: deployed_models/trading_signal_classifier/1.0.0/trading_signal_classifier_1.0.0.joblib
2026-01-24 02:15:17,162 - TradingModelDeployment - INFO - Checksum: e34d8bc09532bf5d...



📁 Model saved to: deployed_models/trading_signal_classifier/1.0.0/trading_signal_classifier_1.0.0.joblib
📋 Metadata:
{
  "model_name": "trading_signal_classifier",
  "version": "1.0.0",
  "created_at": "2026-01-24T02:15:17.161911",
  "model_type": "Pipeline",
  "feature_names": [
    "returns",
    "volatility_20",
    "momentum_10",
    "momentum_20",
    "rsi",
    "volume_ratio",
    "ma_crossover",
    "spread",
    "hour",
    "day_of_week"
  ],
  "target_name": "target",
  "training_metrics": {
    "accuracy": 0.504,
    "precision": 0.468,
    "recall": 0.24375,
    "f1_score": 0.32054794520547947
  },
  "hyperparameters": {
    "n_estimators": 100,
    "max_depth": 10,
    "min_samples_split": 20
  },
  "checksum": "e34d8bc09532bf5dbe3fa75328a58028a2666acfb3c7d766dd6421fcbe349a4e",
  "description": "Random Forest classifier for trading signal prediction",
  "author": "Quant Team"
}


In [6]:
# Load and verify the model
loaded_model, loaded_metadata = serializer.load_model(
    model_name="trading_signal_classifier",
    version="1.0.0",
    verify_checksum=True
)

# Test that loaded model works correctly
test_predictions = loaded_model.predict(X_test[:5])
print(f"\n✅ Model loaded and verified successfully")
print(f"   Test predictions: {test_predictions}")
print(f"   Model type: {loaded_metadata.model_type}")
print(f"   Features: {loaded_metadata.feature_names}")

2026-01-24 02:15:17,299 - TradingModelDeployment - INFO - ✅ Checksum verified
2026-01-24 02:15:17,310 - TradingModelDeployment - INFO - Model loaded: trading_signal_classifier v1.0.0



✅ Model loaded and verified successfully
   Test predictions: [0 0 0 0 0]
   Model type: Pipeline
   Features: ['returns', 'volatility_20', 'momentum_10', 'momentum_20', 'rsi', 'volume_ratio', 'ma_crossover', 'spread', 'hour', 'day_of_week']


## 4. Build a Prediction API with Flask

This section demonstrates how to create a production-ready REST API for serving trading model predictions. The API includes health checks, input validation, and proper error handling.

In [7]:
class TradingModelAPI:
    """
    Production-ready API wrapper for trading model predictions.
    
    This class simulates a Flask/FastAPI service structure.
    In production, this would be split into separate files.
    
    Endpoints:
    - /health: Health check
    - /predict: Single prediction
    - /predict_batch: Batch predictions
    - /model_info: Model metadata
    """
    
    def __init__(
        self,
        model: Any,
        metadata: ModelMetadata,
        max_batch_size: int = 1000
    ):
        self.model = model
        self.metadata = metadata
        self.max_batch_size = max_batch_size
        self.request_count = 0
        self.total_latency = 0.0
        self.startup_time = datetime.now()
        
        logger.info(f"API initialized with model: {metadata.model_name} v{metadata.version}")
    
    def health_check(self) -> Dict:
        """
        Health check endpoint.
        Returns service status and basic metrics.
        """
        uptime = (datetime.now() - self.startup_time).total_seconds()
        avg_latency = self.total_latency / max(self.request_count, 1) * 1000
        
        return {
            "status": "healthy",
            "model_name": self.metadata.model_name,
            "model_version": self.metadata.version,
            "uptime_seconds": round(uptime, 2),
            "requests_served": self.request_count,
            "avg_latency_ms": round(avg_latency, 2),
            "timestamp": datetime.now().isoformat()
        }
    
    def model_info(self) -> Dict:
        """
        Model information endpoint.
        Returns model metadata and feature requirements.
        """
        return {
            "model_name": self.metadata.model_name,
            "version": self.metadata.version,
            "model_type": self.metadata.model_type,
            "created_at": self.metadata.created_at,
            "feature_names": self.metadata.feature_names,
            "required_features": len(self.metadata.feature_names),
            "training_metrics": self.metadata.training_metrics,
            "description": self.metadata.description
        }
    
    def validate_input(self, features: Dict) -> Tuple[bool, str]:
        """Validate input features against model requirements."""
        missing_features = []
        for feature_name in self.metadata.feature_names:
            if feature_name not in features:
                missing_features.append(feature_name)
        
        if missing_features:
            return False, f"Missing features: {missing_features}"
        
        # Check for valid numeric values
        for feature_name, value in features.items():
            if feature_name in self.metadata.feature_names:
                if not isinstance(value, (int, float)):
                    return False, f"Feature '{feature_name}' must be numeric, got {type(value)}"
                if np.isnan(value) or np.isinf(value):
                    return False, f"Feature '{feature_name}' contains invalid value: {value}"
        
        return True, "Valid"
    
    def predict(self, features: Dict) -> Dict:
        """
        Single prediction endpoint.
        
        Parameters:
        -----------
        features : dict with feature_name: value pairs
        
        Returns:
        --------
        Dict with prediction, probability, and metadata
        """
        start_time = time.perf_counter()
        
        # Validate input
        is_valid, message = self.validate_input(features)
        if not is_valid:
            return {
                "status": "error",
                "error": message,
                "timestamp": datetime.now().isoformat()
            }
        
        try:
            # Prepare features array in correct order
            feature_array = np.array([
                [features[name] for name in self.metadata.feature_names]
            ])
            
            # Get prediction and probability
            prediction = self.model.predict(feature_array)[0]
            
            # Get probability if available
            if hasattr(self.model, 'predict_proba'):
                probabilities = self.model.predict_proba(feature_array)[0]
                confidence = float(max(probabilities))
            else:
                probabilities = None
                confidence = None
            
            # Calculate latency
            latency = time.perf_counter() - start_time
            self.request_count += 1
            self.total_latency += latency
            
            return {
                "status": "success",
                "prediction": int(prediction),
                "signal": "BUY" if prediction == 1 else "HOLD",
                "confidence": confidence,
                "probabilities": {
                    "class_0": float(probabilities[0]) if probabilities is not None else None,
                    "class_1": float(probabilities[1]) if probabilities is not None else None
                },
                "latency_ms": round(latency * 1000, 3),
                "model_version": self.metadata.version,
                "timestamp": datetime.now().isoformat()
            }
            
        except Exception as e:
            logger.error(f"Prediction error: {str(e)}")
            return {
                "status": "error",
                "error": str(e),
                "timestamp": datetime.now().isoformat()
            }
    
    def predict_batch(self, features_list: List[Dict]) -> Dict:
        """
        Batch prediction endpoint.
        
        Parameters:
        -----------
        features_list : list of feature dicts
        
        Returns:
        --------
        Dict with list of predictions and batch metadata
        """
        start_time = time.perf_counter()
        
        # Check batch size
        if len(features_list) > self.max_batch_size:
            return {
                "status": "error",
                "error": f"Batch size {len(features_list)} exceeds maximum {self.max_batch_size}",
                "timestamp": datetime.now().isoformat()
            }
        
        if len(features_list) == 0:
            return {
                "status": "error",
                "error": "Empty batch received",
                "timestamp": datetime.now().isoformat()
            }
        
        try:
            # Validate all inputs
            for i, features in enumerate(features_list):
                is_valid, message = self.validate_input(features)
                if not is_valid:
                    return {
                        "status": "error",
                        "error": f"Invalid input at index {i}: {message}",
                        "timestamp": datetime.now().isoformat()
                    }
            
            # Prepare batch array
            feature_array = np.array([
                [features[name] for name in self.metadata.feature_names]
                for features in features_list
            ])
            
            # Batch prediction
            predictions = self.model.predict(feature_array)
            
            if hasattr(self.model, 'predict_proba'):
                probabilities = self.model.predict_proba(feature_array)
            else:
                probabilities = None
            
            # Format results
            results = []
            for i, pred in enumerate(predictions):
                result = {
                    "index": i,
                    "prediction": int(pred),
                    "signal": "BUY" if pred == 1 else "HOLD"
                }
                if probabilities is not None:
                    result["confidence"] = float(max(probabilities[i]))
                results.append(result)
            
            latency = time.perf_counter() - start_time
            self.request_count += len(features_list)
            self.total_latency += latency
            
            return {
                "status": "success",
                "batch_size": len(features_list),
                "predictions": results,
                "total_latency_ms": round(latency * 1000, 3),
                "avg_latency_per_sample_ms": round(latency * 1000 / len(features_list), 3),
                "model_version": self.metadata.version,
                "timestamp": datetime.now().isoformat()
            }
            
        except Exception as e:
            logger.error(f"Batch prediction error: {str(e)}")
            return {
                "status": "error",
                "error": str(e),
                "timestamp": datetime.now().isoformat()
            }

# Initialize API with our model
api = TradingModelAPI(
    model=loaded_model,
    metadata=loaded_metadata
)
print("✅ Trading Model API initialized")

2026-01-24 02:15:17,377 - TradingModelDeployment - INFO - API initialized with model: trading_signal_classifier v1.0.0


✅ Trading Model API initialized


In [8]:
# Test the API endpoints

# 1. Health Check
print("=" * 60)
print("🏥 HEALTH CHECK")
print("=" * 60)
health = api.health_check()
print(json.dumps(health, indent=2))

# 2. Model Info
print("\n" + "=" * 60)
print("📋 MODEL INFO")
print("=" * 60)
info = api.model_info()
print(json.dumps(info, indent=2))

🏥 HEALTH CHECK
{
  "status": "healthy",
  "model_name": "trading_signal_classifier",
  "model_version": "1.0.0",
  "uptime_seconds": 0.1,
  "requests_served": 0,
  "avg_latency_ms": 0.0,
  "timestamp": "2026-01-24T02:15:17.482575"
}

📋 MODEL INFO
{
  "model_name": "trading_signal_classifier",
  "version": "1.0.0",
  "model_type": "Pipeline",
  "created_at": "2026-01-24T02:15:17.161911",
  "feature_names": [
    "returns",
    "volatility_20",
    "momentum_10",
    "momentum_20",
    "rsi",
    "volume_ratio",
    "ma_crossover",
    "spread",
    "hour",
    "day_of_week"
  ],
  "required_features": 10,
  "training_metrics": {
    "accuracy": 0.504,
    "precision": 0.468,
    "recall": 0.24375,
    "f1_score": 0.32054794520547947
  },
  "description": "Random Forest classifier for trading signal prediction"
}


In [9]:
# 3. Single Prediction
print("=" * 60)
print("🎯 SINGLE PREDICTION")
print("=" * 60)

sample_features = {
    'returns': 0.005,
    'volatility_20': 0.02,
    'momentum_10': 0.03,
    'momentum_20': 0.05,
    'rsi': 65.0,
    'volume_ratio': 1.2,
    'ma_crossover': 1,
    'spread': 0.0005,
    'hour': 10,
    'day_of_week': 2
}

prediction = api.predict(sample_features)
print(json.dumps(prediction, indent=2))

# 4. Batch Prediction
print("\n" + "=" * 60)
print("📦 BATCH PREDICTION")
print("=" * 60)

batch_features = [
    {'returns': 0.005, 'volatility_20': 0.02, 'momentum_10': 0.03, 'momentum_20': 0.05,
     'rsi': 65.0, 'volume_ratio': 1.2, 'ma_crossover': 1, 'spread': 0.0005, 'hour': 10, 'day_of_week': 2},
    {'returns': -0.01, 'volatility_20': 0.03, 'momentum_10': -0.02, 'momentum_20': -0.03,
     'rsi': 35.0, 'volume_ratio': 0.8, 'ma_crossover': 0, 'spread': 0.001, 'hour': 14, 'day_of_week': 4},
    {'returns': 0.002, 'volatility_20': 0.015, 'momentum_10': 0.01, 'momentum_20': 0.02,
     'rsi': 55.0, 'volume_ratio': 1.0, 'ma_crossover': 1, 'spread': 0.0003, 'hour': 11, 'day_of_week': 1},
]

batch_result = api.predict_batch(batch_features)
print(json.dumps(batch_result, indent=2))

🎯 SINGLE PREDICTION
{
  "status": "success",
  "prediction": 0,
  "signal": "HOLD",
  "confidence": 0.5709211661330337,
  "probabilities": {
    "class_0": 0.5709211661330337,
    "class_1": 0.4290788338669664
  },
  "latency_ms": 29.442,
  "model_version": "1.0.0",
  "timestamp": "2026-01-24T02:15:17.619410"
}

📦 BATCH PREDICTION
{
  "status": "success",
  "batch_size": 3,
  "predictions": [
    {
      "index": 0,
      "prediction": 0,
      "signal": "HOLD",
      "confidence": 0.5709211661330337
    },
    {
      "index": 1,
      "prediction": 0,
      "signal": "HOLD",
      "confidence": 0.5780127103813741
    },
    {
      "index": 2,
      "prediction": 0,
      "signal": "HOLD",
      "confidence": 0.507961695494945
    }
  ],
  "total_latency_ms": 27.566,
  "avg_latency_per_sample_ms": 9.189,
  "model_version": "1.0.0",
  "timestamp": "2026-01-24T02:15:17.647279"
}


### Flask API Template

Here's a complete Flask application template for production deployment:

In [10]:
# Flask API Template (save as app.py for production deployment)

flask_app_template = '''
"""
Trading Model API - Flask Application

Usage:
    pip install flask gunicorn
    gunicorn -w 4 -b 0.0.0.0:5000 app:app
"""

from flask import Flask, request, jsonify
import joblib
import json
import numpy as np
import time
from datetime import datetime
import logging

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

app = Flask(__name__)

# Load model and metadata at startup
MODEL = None
METADATA = None
REQUEST_COUNT = 0
STARTUP_TIME = datetime.now()

def load_model():
    """Load model and metadata on startup."""
    global MODEL, METADATA
    MODEL = joblib.load('./deployed_models/trading_signal_classifier/1.0.0/trading_signal_classifier_1.0.0.joblib')
    with open('./deployed_models/trading_signal_classifier/1.0.0/metadata.json', 'r') as f:
        METADATA = json.load(f)
    logger.info(f"Model loaded: {METADATA['model_name']} v{METADATA['version']}")

@app.route('/health', methods=['GET'])
def health():
    """Health check endpoint."""
    uptime = (datetime.now() - STARTUP_TIME).total_seconds()
    return jsonify({
        'status': 'healthy',
        'model_name': METADATA['model_name'],
        'model_version': METADATA['version'],
        'uptime_seconds': round(uptime, 2),
        'requests_served': REQUEST_COUNT,
        'timestamp': datetime.now().isoformat()
    })

@app.route('/predict', methods=['POST'])
def predict():
    """Single prediction endpoint."""
    global REQUEST_COUNT
    start_time = time.perf_counter()
    
    try:
        features = request.json
        
        # Validate features
        feature_names = METADATA['feature_names']
        for name in feature_names:
            if name not in features:
                return jsonify({'status': 'error', 'error': f'Missing feature: {name}'}), 400
        
        # Prepare feature array
        feature_array = np.array([[features[name] for name in feature_names]])
        
        # Predict
        prediction = MODEL.predict(feature_array)[0]
        probabilities = MODEL.predict_proba(feature_array)[0]
        
        REQUEST_COUNT += 1
        latency = time.perf_counter() - start_time
        
        return jsonify({
            'status': 'success',
            'prediction': int(prediction),
            'signal': 'BUY' if prediction == 1 else 'HOLD',
            'confidence': float(max(probabilities)),
            'latency_ms': round(latency * 1000, 3),
            'timestamp': datetime.now().isoformat()
        })
        
    except Exception as e:
        logger.error(f"Prediction error: {str(e)}")
        return jsonify({'status': 'error', 'error': str(e)}), 500

@app.route('/predict_batch', methods=['POST'])
def predict_batch():
    """Batch prediction endpoint."""
    global REQUEST_COUNT
    start_time = time.perf_counter()
    
    try:
        features_list = request.json
        
        if len(features_list) > 1000:
            return jsonify({'status': 'error', 'error': 'Batch size exceeds limit'}), 400
        
        feature_names = METADATA['feature_names']
        feature_array = np.array([
            [f[name] for name in feature_names] for f in features_list
        ])
        
        predictions = MODEL.predict(feature_array)
        probabilities = MODEL.predict_proba(feature_array)
        
        results = [
            {'index': i, 'prediction': int(p), 'signal': 'BUY' if p == 1 else 'HOLD',
             'confidence': float(max(proba))}
            for i, (p, proba) in enumerate(zip(predictions, probabilities))
        ]
        
        REQUEST_COUNT += len(features_list)
        latency = time.perf_counter() - start_time
        
        return jsonify({
            'status': 'success',
            'batch_size': len(features_list),
            'predictions': results,
            'total_latency_ms': round(latency * 1000, 3),
            'timestamp': datetime.now().isoformat()
        })
        
    except Exception as e:
        logger.error(f"Batch prediction error: {str(e)}")
        return jsonify({'status': 'error', 'error': str(e)}), 500

@app.route('/model_info', methods=['GET'])
def model_info():
    """Model information endpoint."""
    return jsonify({
        'model_name': METADATA['model_name'],
        'version': METADATA['version'],
        'model_type': METADATA['model_type'],
        'feature_names': METADATA['feature_names'],
        'training_metrics': METADATA['training_metrics']
    })

if __name__ == '__main__':
    load_model()
    app.run(host='0.0.0.0', port=5000, debug=False)
'''

print("📝 Flask API Template")
print("=" * 60)
print(flask_app_template)

📝 Flask API Template

"""
Trading Model API - Flask Application

Usage:
    pip install flask gunicorn
    gunicorn -w 4 -b 0.0.0.0:5000 app:app
"""

from flask import Flask, request, jsonify
import joblib
import json
import numpy as np
import time
from datetime import datetime
import logging

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

app = Flask(__name__)

# Load model and metadata at startup
MODEL = None
METADATA = None
REQUEST_COUNT = 0
STARTUP_TIME = datetime.now()

def load_model():
    """Load model and metadata on startup."""
    global MODEL, METADATA
    MODEL = joblib.load('./deployed_models/trading_signal_classifier/1.0.0/trading_signal_classifier_1.0.0.joblib')
    with open('./deployed_models/trading_signal_classifier/1.0.0/metadata.json', 'r') as f:
        METADATA = json.load(f)
    logger.info(f"Model loaded: {METADATA['model_name']} v{METADATA['version']}")

@app.route('/health', methods=['GET'])
def health():
  

## 5. Implement Real-time Data Pipeline

Real-time prediction pipelines are critical for trading systems. This section implements a low-latency pipeline for continuous market data processing.

In [11]:
class RealTimePredictionPipeline:
    """
    Real-time prediction pipeline for trading models.
    
    Features:
    - Sliding window feature computation
    - Low-latency predictions
    - Automatic feature engineering
    - Signal generation with confidence thresholds
    """
    
    def __init__(
        self,
        model: Any,
        feature_names: List[str],
        window_size: int = 50,
        confidence_threshold: float = 0.6
    ):
        self.model = model
        self.feature_names = feature_names
        self.window_size = window_size
        self.confidence_threshold = confidence_threshold
        
        # Data buffers using deque for efficient sliding window
        self.price_buffer = deque(maxlen=window_size)
        self.volume_buffer = deque(maxlen=window_size)
        self.timestamp_buffer = deque(maxlen=window_size)
        
        # Performance tracking
        self.prediction_latencies = deque(maxlen=1000)
        self.signals_generated = 0
        
        logger.info(f"RealTimePipeline initialized with window_size={window_size}")
    
    def compute_features(self, current_data: Dict) -> Optional[Dict]:
        """
        Compute features from current market data and historical buffer.
        
        Parameters:
        -----------
        current_data : dict with 'price', 'volume', 'timestamp', 'spread'
        
        Returns:
        --------
        Dict of computed features or None if insufficient data
        """
        # Add to buffers
        self.price_buffer.append(current_data['price'])
        self.volume_buffer.append(current_data['volume'])
        self.timestamp_buffer.append(current_data['timestamp'])
        
        # Need enough history for feature computation
        if len(self.price_buffer) < 20:
            return None
        
        prices = np.array(self.price_buffer)
        volumes = np.array(self.volume_buffer)
        
        # Compute returns
        returns = np.diff(prices) / prices[:-1]
        current_return = returns[-1] if len(returns) > 0 else 0
        
        # Features
        features = {
            'returns': current_return,
            'volatility_20': np.std(returns[-20:]) if len(returns) >= 20 else np.std(returns),
            'momentum_10': (prices[-1] / prices[-10] - 1) if len(prices) >= 10 else 0,
            'momentum_20': (prices[-1] / prices[-20] - 1) if len(prices) >= 20 else 0,
            'rsi': self._compute_rsi(returns[-14:]) if len(returns) >= 14 else 50,
            'volume_ratio': volumes[-1] / np.mean(volumes[-20:]) if len(volumes) >= 20 else 1.0,
            'ma_crossover': 1 if np.mean(prices[-20:]) > np.mean(prices[-50:]) else 0 if len(prices) >= 50 else 0,
            'spread': current_data.get('spread', 0.0005),
            'hour': current_data['timestamp'].hour,
            'day_of_week': current_data['timestamp'].weekday()
        }
        
        return features
    
    def _compute_rsi(self, returns: np.ndarray, period: int = 14) -> float:
        """Compute Relative Strength Index."""
        gains = np.maximum(returns, 0)
        losses = np.abs(np.minimum(returns, 0))
        
        avg_gain = np.mean(gains)
        avg_loss = np.mean(losses)
        
        if avg_loss == 0:
            return 100.0
        
        rs = avg_gain / avg_loss
        rsi = 100 - (100 / (1 + rs))
        
        return float(rsi)
    
    def predict(self, current_data: Dict) -> Optional[Dict]:
        """
        Generate prediction from current market data.
        
        Parameters:
        -----------
        current_data : dict with market data
        
        Returns:
        --------
        Dict with signal or None if insufficient data
        """
        start_time = time.perf_counter()
        
        # Compute features
        features = self.compute_features(current_data)
        
        if features is None:
            return {
                'status': 'warming_up',
                'buffer_size': len(self.price_buffer),
                'required': 20
            }
        
        try:
            # Prepare feature array
            feature_array = np.array([
                [features[name] for name in self.feature_names]
            ])
            
            # Get prediction and probability
            prediction = self.model.predict(feature_array)[0]
            probabilities = self.model.predict_proba(feature_array)[0]
            confidence = float(max(probabilities))
            
            # Generate signal based on confidence threshold
            if confidence >= self.confidence_threshold:
                signal = "BUY" if prediction == 1 else "SELL"
                self.signals_generated += 1
            else:
                signal = "HOLD"
            
            latency = time.perf_counter() - start_time
            self.prediction_latencies.append(latency)
            
            return {
                'status': 'success',
                'timestamp': current_data['timestamp'].isoformat(),
                'price': current_data['price'],
                'prediction': int(prediction),
                'signal': signal,
                'confidence': round(confidence, 4),
                'features': {k: round(v, 6) if isinstance(v, float) else v 
                           for k, v in features.items()},
                'latency_us': round(latency * 1_000_000, 2)  # microseconds
            }
            
        except Exception as e:
            logger.error(f"Prediction error: {str(e)}")
            return {
                'status': 'error',
                'error': str(e)
            }
    
    def get_stats(self) -> Dict:
        """Get pipeline statistics."""
        latencies = list(self.prediction_latencies)
        
        if not latencies:
            return {'status': 'no_predictions'}
        
        return {
            'total_predictions': len(latencies),
            'signals_generated': self.signals_generated,
            'avg_latency_us': round(np.mean(latencies) * 1_000_000, 2),
            'p50_latency_us': round(np.percentile(latencies, 50) * 1_000_000, 2),
            'p95_latency_us': round(np.percentile(latencies, 95) * 1_000_000, 2),
            'p99_latency_us': round(np.percentile(latencies, 99) * 1_000_000, 2),
            'buffer_size': len(self.price_buffer)
        }

# Initialize pipeline
pipeline = RealTimePredictionPipeline(
    model=loaded_model,
    feature_names=FEATURE_COLUMNS,
    window_size=100,
    confidence_threshold=0.6
)
print("✅ Real-time Pipeline initialized")

2026-01-24 02:15:17,822 - TradingModelDeployment - INFO - RealTimePipeline initialized with window_size=100


✅ Real-time Pipeline initialized


In [12]:
# Simulate real-time data stream
def simulate_market_tick() -> Dict:
    """Simulate a single market data tick."""
    base_price = 100 + np.random.randn() * 2
    return {
        'price': base_price + np.random.randn() * 0.5,
        'volume': np.random.exponential(1000000),
        'spread': np.random.exponential(0.0005),
        'timestamp': datetime.now()
    }

# Run simulation
print("=" * 60)
print("📡 REAL-TIME PIPELINE SIMULATION")
print("=" * 60)

results = []
for i in range(100):
    tick = simulate_market_tick()
    result = pipeline.predict(tick)
    results.append(result)
    
    # Print significant signals
    if result.get('signal') in ['BUY', 'SELL']:
        print(f"🚨 Signal: {result['signal']} at {result['price']:.2f} "
              f"(conf: {result['confidence']:.2%}) - Latency: {result['latency_us']:.0f}µs")

# Print statistics
print("\n" + "=" * 60)
print("📊 PIPELINE STATISTICS")
print("=" * 60)
stats = pipeline.get_stats()
print(json.dumps(stats, indent=2))

📡 REAL-TIME PIPELINE SIMULATION
🚨 Signal: SELL at 99.75 (conf: 62.35%) - Latency: 28306µs

📊 PIPELINE STATISTICS
{
  "total_predictions": 81,
  "signals_generated": 1,
  "avg_latency_us": 28536.03,
  "p50_latency_us": 27986.83,
  "p95_latency_us": 39488.75,
  "p99_latency_us": 43854.83,
  "buffer_size": 100
}


## 6. Create Batch Prediction System

Batch prediction systems are used for end-of-day processing, backtesting, and bulk signal generation.

In [13]:
class BatchPredictionSystem:
    """
    Efficient batch prediction system for trading models.
    
    Features:
    - Chunked processing for memory efficiency
    - Progress tracking
    - Parallel processing support
    - Result aggregation and statistics
    """
    
    def __init__(
        self,
        model: Any,
        feature_names: List[str],
        chunk_size: int = 10000
    ):
        self.model = model
        self.feature_names = feature_names
        self.chunk_size = chunk_size
        
        logger.info(f"BatchPredictionSystem initialized with chunk_size={chunk_size}")
    
    def predict_dataframe(
        self,
        df: pd.DataFrame,
        output_file: Optional[str] = None,
        include_probabilities: bool = True
    ) -> pd.DataFrame:
        """
        Generate predictions for entire DataFrame.
        
        Parameters:
        -----------
        df : DataFrame with feature columns
        output_file : optional path to save results
        include_probabilities : whether to include probability scores
        
        Returns:
        --------
        DataFrame with predictions added
        """
        start_time = time.time()
        n_samples = len(df)
        n_chunks = (n_samples + self.chunk_size - 1) // self.chunk_size
        
        logger.info(f"Processing {n_samples} samples in {n_chunks} chunks")
        
        # Validate features
        missing_features = set(self.feature_names) - set(df.columns)
        if missing_features:
            raise ValueError(f"Missing features: {missing_features}")
        
        # Initialize result arrays
        all_predictions = []
        all_probabilities_0 = []
        all_probabilities_1 = []
        
        # Process in chunks
        for i, chunk_start in enumerate(range(0, n_samples, self.chunk_size)):
            chunk_end = min(chunk_start + self.chunk_size, n_samples)
            chunk_df = df.iloc[chunk_start:chunk_end]
            
            # Extract features
            X_chunk = chunk_df[self.feature_names].values
            
            # Predict
            predictions = self.model.predict(X_chunk)
            all_predictions.extend(predictions)
            
            if include_probabilities and hasattr(self.model, 'predict_proba'):
                probabilities = self.model.predict_proba(X_chunk)
                all_probabilities_0.extend(probabilities[:, 0])
                all_probabilities_1.extend(probabilities[:, 1])
            
            # Progress update
            progress = (i + 1) / n_chunks * 100
            if (i + 1) % max(1, n_chunks // 10) == 0:
                logger.info(f"Progress: {progress:.1f}% ({chunk_end}/{n_samples})")
        
        # Create result DataFrame
        result_df = df.copy()
        result_df['prediction'] = all_predictions
        result_df['signal'] = result_df['prediction'].map({1: 'BUY', 0: 'HOLD'})
        
        if include_probabilities and all_probabilities_0:
            result_df['prob_class_0'] = all_probabilities_0
            result_df['prob_class_1'] = all_probabilities_1
            result_df['confidence'] = result_df[['prob_class_0', 'prob_class_1']].max(axis=1)
        
        # Calculate processing time
        elapsed = time.time() - start_time
        throughput = n_samples / elapsed
        
        logger.info(f"Batch prediction complete: {n_samples} samples in {elapsed:.2f}s "
                   f"({throughput:.0f} samples/sec)")
        
        # Save if output file specified
        if output_file:
            result_df.to_parquet(output_file, index=False)
            logger.info(f"Results saved to {output_file}")
        
        return result_df
    
    def generate_signal_report(self, result_df: pd.DataFrame) -> Dict:
        """Generate summary report of batch predictions."""
        total = len(result_df)
        buy_signals = (result_df['signal'] == 'BUY').sum()
        hold_signals = (result_df['signal'] == 'HOLD').sum()
        
        report = {
            'total_samples': total,
            'buy_signals': int(buy_signals),
            'hold_signals': int(hold_signals),
            'buy_percentage': round(buy_signals / total * 100, 2),
            'prediction_distribution': result_df['prediction'].value_counts().to_dict()
        }
        
        if 'confidence' in result_df.columns:
            report['confidence_stats'] = {
                'mean': round(result_df['confidence'].mean(), 4),
                'std': round(result_df['confidence'].std(), 4),
                'min': round(result_df['confidence'].min(), 4),
                'max': round(result_df['confidence'].max(), 4),
                'high_confidence_count': int((result_df['confidence'] >= 0.7).sum())
            }
        
        return report

# Initialize batch system
batch_system = BatchPredictionSystem(
    model=loaded_model,
    feature_names=FEATURE_COLUMNS,
    chunk_size=5000
)
print("✅ Batch Prediction System initialized")

2026-01-24 02:15:20,561 - TradingModelDeployment - INFO - BatchPredictionSystem initialized with chunk_size=5000


✅ Batch Prediction System initialized


In [14]:
# Run batch prediction on test data
print("=" * 60)
print("📦 BATCH PREDICTION DEMONSTRATION")
print("=" * 60)

# Create test DataFrame
test_df = pd.DataFrame(X_test, columns=FEATURE_COLUMNS)
test_df['actual'] = y_test

# Run batch prediction
results_df = batch_system.predict_dataframe(test_df, include_probabilities=True)

# Generate report
print("\n" + "=" * 60)
print("📊 SIGNAL REPORT")
print("=" * 60)
report = batch_system.generate_signal_report(results_df)
print(json.dumps(report, indent=2))

# Show sample results
print("\n📋 Sample Results:")
results_df[['returns', 'rsi', 'prediction', 'signal', 'confidence', 'actual']].head(10)

2026-01-24 02:15:20,580 - TradingModelDeployment - INFO - Processing 2000 samples in 1 chunks
2026-01-24 02:15:20,623 - TradingModelDeployment - INFO - Progress: 100.0% (2000/2000)
2026-01-24 02:15:20,627 - TradingModelDeployment - INFO - Batch prediction complete: 2000 samples in 0.05s (43168 samples/sec)


📦 BATCH PREDICTION DEMONSTRATION

📊 SIGNAL REPORT
{
  "total_samples": 2000,
  "buy_signals": 500,
  "hold_signals": 1500,
  "buy_percentage": 25.0,
  "prediction_distribution": {
    "0": 1500,
    "1": 500
  },
  "confidence_stats": {
    "mean": 0.5317,
    "std": 0.026,
    "min": 0.5,
    "max": 0.6923,
    "high_confidence_count": 0
  }
}

📋 Sample Results:


,returns,rsi,prediction,signal,confidence,actual
0,0.052758,64.505788,0,HOLD,0.592586,1
1,-0.020741,43.865324,0,HOLD,0.510596,1
2,-0.006986,47.907503,0,HOLD,0.518321,0
3,0.000032,50.009714,0,HOLD,0.505610,1
4,0.021487,56.348597,0,HOLD,0.538585,1
5,-0.018518,44.507301,1,BUY,0.512731,0
6,0.004712,51.412642,0,HOLD,0.526660,1
7,0.002989,50.896470,0,HOLD,0.525755,1
8,0.013664,54.073951,0,HOLD,0.576430,1
9,-0.001111,49.666728,1,BUY,0.506601,1


## 7. Build Model Monitoring and Logging

Comprehensive monitoring is essential for detecting model drift, performance degradation, and system issues in production.